In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

#Jets adding
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

In [4]:
#import combined data from csv
#data = pd.read_csv("E:\\6242Project\\FeaturesDataset\\philly_property_data.csv")
#data = pd.read_csv("E:\\6242Project\\FeaturesDataset\\washington_property_data.csv")
data = pd.read_csv("E:\\6242Project\\FeaturesDataset\\philly_enriched.csv")



In [5]:
y = data['sale_price']
data_Features = data.drop(columns=['market_value', 'sale_price'])
data_hot = data_Features.drop(columns=['sale_date'])
#data_hot = data_Features.drop(columns=['sale_date','view_scale','heater_scale','central_air','garage_spaces','has_central_air', 'has_type_heater'])
data_hot = pd.get_dummies(data_hot, drop_first=True)


In [6]:
#train test split
#X_train, X_test, y_train, y_test = train_test_split(data10_hot, y, test_size=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(data_hot, y, test_size=0.3, random_state=1)

# Standardize full feature set for Elastic Net variable selection
scaler_full = StandardScaler()
X_train_scaled_full = scaler_full.fit_transform(X_train)
X_test_scaled_full = scaler_full.transform(X_test)

Selected features:
Index(['zip_code', 'view_scale', 'heater_scale', 'exterior_condition',
       'interior_condition', 'central_air', 'basement_scale', 'garage_spaces',
       'number_of_bedrooms', 'number_of_bathrooms', 'number_stories',
       'total_livable_area', 'total_area', 'year_built', 'year_built_estimate',
       'has_basements', 'has_central_air', 'has_type_heater',
       'is_nominal_sale', 'active_inventory', 'new_listings', 'pending_sales',
       'homes_sold', 'sale_to_list_ratio', 'percent_above_list',
       'avg_days_on_market', 'avg_list_price_per_sqft', 'avg_listed_price',
       'avg_price_per_sqft', 'avg_price_sold', 'property_type_Condo/Co-op',
       'property_type_Single Family Residential', 'property_type_Townhouse'],
      dtype='object')


In [ ]:
# Variable selection with Elastic Net
elasticNet = ElasticNetCV(l1_ratio=0.5, cv=5)
elasticNet.fit(X_train_scaled_full, y_train)

# Get the selected features
coefficients = elasticNet.coef_
selected_features = data_hot.columns[coefficients != 0]
print("Selected features:")
print(selected_features)

In [7]:
print("--------------------------------------End of Variable Selection--------------------------------------")

--------------------------------------End of Variable Selection--------------------------------------


In [8]:
#Instiantiate model
LMmodel = LinearRegression()

#fit model to training data. Using only selected features
X_train_selected = X_train[selected_features]

LMmodel.fit(X_train_selected, y_train)

#predict on test data
x_test_selected = X_test[selected_features]
y_pred = LMmodel.predict(x_test_selected)

print("Predictions:")
print(y_pred)

#calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred)
print(R2)

#Access coefficients
coefficients = LMmodel.coef_
print("Coefficients:")
print(coefficients)

#Access intercept
intercept = LMmodel.intercept_
print("Intercept:")
print(intercept)

Predictions:
[-537369.20066797  152133.08824883  946884.31728987 ... 1837064.34011057
  -49503.69635534  860918.09477678]
RMSE:
952100.9473387457
R2:
0.2843833569632681
Coefficients:
[-1.20206115e+03 -3.58496724e+03  2.26321415e+02  2.81578734e+03
 -1.28579135e+04  1.12926048e+05  2.64005110e+04  1.61890279e+05
  2.52747589e+04  1.03306658e+05  6.44274137e+05  1.25791200e+01
  1.06244987e-02 -7.61481362e+01 -1.91082393e+04 -4.99086230e+05
 -3.74645340e+04 -1.41048445e+05 -9.05862581e+03  3.12105133e+03
 -4.82775977e+03 -2.41868858e+03  5.08989008e+03 -6.12531503e+05
 -1.46920663e+05 -3.33050835e+02  4.92639757e+02  3.39880518e-01
  4.20997093e+02 -4.96525244e-01 -2.84614134e+05 -3.36402705e+05
 -2.89734680e+05]
Intercept:
23207357.459598914


In [9]:
#Instantiating Random Forest Model
RFmodel = RandomForestRegressor(n_estimators=50, random_state=1,n_jobs=-1 ,verbose=1)

#RF model fit
RFmodel.fit(X_train, y_train)

#Sanity check
print("Progress...")

#Predicting on test data
y_pred_RF = RFmodel.predict(X_test)
print("Predictions:")
print(y_pred_RF)

# Calculate RMSE and R2
RMSE = np.sqrt(mean_squared_error(y_test, y_pred_RF))
print("RMSE:")
print(RMSE)
print("R2:")
R2 = r2_score(y_test, y_pred_RF)
print(R2)

# Access feature importances (since Random Forest models provide feature importance)
feature_importances = RFmodel.feature_importances_
print("Feature Importances:")
print(feature_importances)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.4s


Progress...
Predictions:
[ 178187.38  149372.38  523496.34 ... 2278351.98  186292.82  438988.12]
RMSE:
549261.7069863403
R2:
0.7618376634963818
Feature Importances:
[1.05245341e-02 1.72335341e-03 9.72178037e-03 3.42494672e-03
 2.75922116e-03 6.98573184e-03 1.45168106e-03 1.10051292e-02
 2.04383758e-02 1.83347753e-02 2.66111683e-01 4.55765098e-01
 4.76672630e-02 3.29111262e-02 5.21622133e-04 4.50485011e-05
 1.01065626e-03 1.49204916e-03 1.17409352e-02 5.07228773e-03
 1.36246022e-02 1.06507196e-02 6.24790669e-03 1.34826292e-02
 7.03387739e-03 1.05362948e-02 0.00000000e+00 3.30729457e-03
 3.89495933e-03 6.01825071e-03 9.83205860e-03 6.16082096e-03
 4.22203152e-05 9.13296501e-05 3.69737348e-04]


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    3.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.0s finished


In [10]:
# Instantiate Gradient Boosting model

'''
GBmodel = GradientBoostingRegressor(
    n_estimators=200,
    learning_rate=0.15,
    max_depth=4,
    random_state=1,
    verbose=1
)
'''
GBmodel = GradientBoostingRegressor(
    subsample= 1.0, n_estimators= 500,
    min_samples_split= 2, min_samples_leaf= 1,
    max_features= None, max_depth= 4, learning_rate= 0.15
)

# Fit the model on untransformed target
GBmodel.fit(X_train, y_train)

# Predict on test data
y_pred_GB = GBmodel.predict(X_test)

# Evaluate performance
RMSE_GB = np.sqrt(mean_squared_error(y_test, y_pred_GB))
R2_GB = r2_score(y_test, y_pred_GB)

print("Predictions:")
print(y_pred_GB)
print("RMSE (Gradient Boosting):", RMSE_GB)
print("R² (Gradient Boosting):", R2_GB)

# Access feature importances
feature_importances_GB = GBmodel.feature_importances_
print("Feature Importances (Gradient Boosting):")
print(feature_importances_GB)

Predictions:
[ 165217.6571103   245633.59760138  490177.32958491 ... 2878432.50511961
  188241.56160294  524439.36327644]
RMSE (Gradient Boosting): 577092.585814359
R² (Gradient Boosting): 0.7370910157270605
Feature Importances (Gradient Boosting):
[9.13267601e-03 6.78343233e-04 5.16700081e-03 2.15967864e-03
 3.34515562e-03 5.73973952e-03 1.58519998e-03 1.25211244e-02
 5.46502468e-03 4.41638026e-02 3.31109998e-01 3.90558620e-01
 5.32379310e-02 2.08319139e-02 3.92311822e-04 5.00139036e-06
 1.67867128e-04 1.60948822e-03 2.95308164e-02 3.71837721e-03
 5.79930285e-03 6.02604908e-03 2.00604553e-03 5.98225667e-03
 1.76158005e-03 2.87801737e-03 0.00000000e+00 8.97368152e-03
 5.77394462e-03 8.28683738e-03 2.38619136e-02 7.49281293e-03
 2.37100112e-06 3.28494145e-05 2.26772608e-06]


In [ ]:
# Define parameter distribution
param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'subsample': [0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Instantiate base model
gbr = GradientBoostingRegressor(random_state=1)

# Set up randomized search
rand_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=50,               # Number of random combinations to try
    scoring='r2',            # Optimize for R² score
    cv=3,                    # 3-fold cross-validation
    verbose=2,
    n_jobs=-1,               # Use all available cores
    random_state=42
)

# Fit to training data
rand_search.fit(X_train, y_train)

# Print best parameters and score
print("Best Parameters Found:")
print(rand_search.best_params_)
print("Best Cross-Validated R² Score:")
print(rand_search.best_score_)

# Evaluate on test set
best_gbr = rand_search.best_estimator_
y_pred_best = best_gbr.predict(X_test)

rmse_best = np.sqrt(mean_squared_error(y_test, y_pred_best))
r2_best = r2_score(y_test, y_pred_best)

print("\nTest Set Evaluation")
print("RMSE:", rmse_best)
print("R²:", r2_best)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
